# 🎓 Tunisia School Success Rate Analysis
## Complete Analysis Notebook (R Kernel)

This notebook contains **all analysis and visualizations** for the Tunisia education data project.

**Workflow:**
1. Run this notebook to perform all analysis
2. Results are saved to `data/` folder (RDS files)
3. Plots are saved to `img/` folder (PNG files)
4. Quarto documents load these pre-computed results

---

## 📦 Section 1: Setup & Load Libraries

In [ ]:
# Setup library path and load packages
.libPaths(Sys.getenv("R_LIBS_USER"))

# Load required libraries
library(tidyverse)
library(readxl)
library(ggplot2)
library(dplyr)
library(knitr)
library(scales)

# Create output directories
dir.create("img", showWarnings = FALSE)
dir.create("data/results", showWarnings = FALSE, recursive = TRUE)

cat("✅ Libraries loaded successfully!\n")
cat("📁 Output directories ready: img/, data/results/")

## 📂 Section 2: Load and Preprocess Data

In [ ]:
# ==============================================================================
# Governorate Mapping (26 code prefixes → 23 governorates)
# ==============================================================================
governorate_mapping <- tibble(
  code_prefix = c("10", "11", "12", "13", "14", "15", "16", "17", "18", "19",
                  "21", "22", "23", "31", "32", "41", "42", "51", "52", "61",
                  "62", "71", "72", "81", "82", "83", "91"),
  governorate = c("Tunis", "Tunis", "Ariana", "Ben Arous", "Manouba", "Nabeul", 
                  "Zaghouan", "Bizerte", "Beja", "Jendouba", "Le Kef", "Siliana",
                  "Sousse", "Monastir", "Mahdia", "Sfax", "Sfax", "Kairouan", 
                  "Gafsa", "Kasserine", "Tataouine", "Gabes", "Medenine", 
                  "Tozeur", "Kebili", "Sidi Bouzid", "Nabeul"),
  governorate_ar = c("تونس", "تونس", "أريانة", "بن عروس", "منوبة", "نابل",
                     "زغوان", "بنزرت", "باجة", "جندوبة", "الكاف", "سليانة",
                     "سوسة", "المنستير", "المهدية", "صفاقس", "صفاقس", "القيروان",
                     "قفصة", "القصرين", "تطاوين", "قابس", "مدنين",
                     "توزر", "قبلي", "سيدي بوزيد", "نابل")
)

cat("✅ Governorate mapping created (26 prefixes → 23 governorates)")

In [ ]:
# ==============================================================================
# Load School Data from Excel
# ==============================================================================
school_data <- read_excel("data/liste_reussite_eleves.xls") %>%
  rename(
    school_code = `Code Etab`,
    school_name = `Nom Etab`,
    level_code = `Code Niveau`,
    level_name = `Nom Niveau`,
    nb_students = `Nbre Eleves`,
    nb_passed = `Nbre Admis`
  ) %>%
  mutate(
    # Calculate success rate
    nb_failed = nb_students - nb_passed,
    success_rate = round(nb_passed / nb_students * 100, 2),
    # Extract code prefix for governorate
    code_prefix = str_sub(as.character(school_code), 1, 2)
  ) %>%
  left_join(governorate_mapping, by = "code_prefix") %>%
  arrange(school_name)

cat("✅ School data loaded:", nrow(school_data), "records\n")
cat("📊 Schools:", n_distinct(school_data$school_code), "\n")
cat("📊 Governorates:", n_distinct(school_data$governorate), "\n")
cat("📊 Education levels:", n_distinct(school_data$level_name))

In [ ]:
# ==============================================================================
# Baccalaureate 2025 Official Results
# ==============================================================================

# Bac success rates by field (شعبة)
bac_by_field <- tibble(
  field_ar = c("رياضيات", "رياضة", "علوم الإعلامية", "العلوم التجريبية", 
               "العلوم التقنية", "الاقتصاد والتصرف", "الآداب"),
  field_en = c("Mathematics", "Sports", "Computer Science", "Experimental Sciences",
               "Technical Sciences", "Economics & Business", "Literature"),
  field_emoji = c("🔢 Mathematics", "🏃 Sports", "💻 Computer Science", "🔬 Experimental Sciences",
                  "⚙️ Technical Sciences", "💼 Economics", "📚 Literature"),
  bac_success_rate = c(74.93, 73.33, 48.47, 45.83, 35.02, 28.95, 23.02)
)

# Bac success rates by governorate
bac_by_governorate <- tibble(
  rank = 1:26,
  governorate_ar = c("صفاقس 1", "مدنين", "صفاقس 2", "سيدي بوزيد", "المهدية", 
                     "سوسة", "أريانة", "المنستير", "تونس 1", "نابل",
                     "بن عروس", "تطاوين", "قابس", "تونس 2", "توزر",
                     "قبلي", "منوبة", "بنزرت", "سليانة", "القصرين",
                     "الكاف", "باجة", "قفصة", "القيروان", "زغوان", "جندوبة"),
  governorate_en = c("Sfax 1", "Medenine", "Sfax 2", "Sidi Bouzid", "Mahdia",
                     "Sousse", "Ariana", "Monastir", "Tunis 1", "Nabeul",
                     "Ben Arous", "Tataouine", "Gabes", "Tunis 2", "Tozeur",
                     "Kebili", "Manouba", "Bizerte", "Siliana", "Kasserine",
                     "Le Kef", "Beja", "Gafsa", "Kairouan", "Zaghouan", "Jendouba"),
  candidates = c(5839, 4964, 4307, 4379, 3988, 7629, 6323, 6928, 5947, 8618,
                 7882, 1420, 4086, 5437, 1423, 2078, 4809, 6767, 2694, 4900,
                 2998, 3541, 4211, 5454, 2119, 4742),
  passed = c(4164, 3423, 2966, 2965, 2540, 4783, 3901, 4251, 3615, 5235,
             4784, 858, 2408, 3152, 819, 1165, 2595, 3634, 1423, 2492,
             1503, 1769, 1956, 2524, 978, 2043),
  bac_success_rate = c(71.31, 68.96, 68.86, 67.71, 63.69, 62.69, 61.70, 61.36, 
                       60.79, 60.74, 60.70, 60.42, 58.93, 57.97, 57.55, 56.06,
                       53.96, 53.70, 52.82, 50.86, 50.13, 49.96, 46.45, 46.28, 
                       46.15, 43.08)
)

cat("✅ Bac 2025 data loaded\n")
cat("📊 Fields:", nrow(bac_by_field), "\n")
cat("📊 Regional delegations:", nrow(bac_by_governorate))

## 📊 Section 3: Exploratory Data Analysis

In [ ]:
# Data structure and sample
cat("📋 DATA STRUCTURE:\n")
cat("═══════════════════════════════════════\n")
glimpse(school_data)

cat("\n\n📋 SAMPLE DATA (first 5 rows):\n")
head(school_data, 5)

In [ ]:
# Summary statistics
cat("📊 OVERALL STATISTICS:\n")
cat("═══════════════════════════════════════\n")

total_students <- sum(school_data$nb_students)
total_passed <- sum(school_data$nb_passed)
overall_rate <- round(total_passed / total_students * 100, 2)

cat("👥 Total Students:", format(total_students, big.mark = ","), "\n")
cat("✅ Total Passed:", format(total_passed, big.mark = ","), "\n")
cat("📈 Overall Success Rate:", overall_rate, "%\n")
cat("🏫 Total Schools:", n_distinct(school_data$school_code), "\n")
cat("🗺️ Governorates:", n_distinct(school_data$governorate), "\n")
cat("📚 Education Levels:", n_distinct(school_data$level_name), "\n")

# Save summary stats
summary_stats <- list(
  total_students = total_students,
  total_passed = total_passed,
  overall_rate = overall_rate,
  n_schools = n_distinct(school_data$school_code),
  n_governorates = n_distinct(school_data$governorate),
  n_levels = n_distinct(school_data$level_name)
)
saveRDS(summary_stats, "data/results/summary_stats.rds")
cat("\n✅ Saved: data/results/summary_stats.rds")

## 📈 Section 4: Statistical Analysis

In [ ]:
# Summary by Governorate
summary_by_gov <- school_data %>%
  filter(!is.na(governorate)) %>%
  group_by(governorate, governorate_ar) %>%
  summarise(
    n_schools = n_distinct(school_code),
    total_students = sum(nb_students),
    total_passed = sum(nb_passed),
    success_rate = round(total_passed / total_students * 100, 2),
    .groups = "drop"
  ) %>%
  arrange(desc(success_rate))

cat("📊 SUCCESS RATE BY GOVERNORATE:\n")
cat("═══════════════════════════════════════\n")
print(summary_by_gov)

# Save for Quarto
saveRDS(summary_by_gov, "data/results/summary_by_governorate.rds")
cat("\n✅ Saved: data/results/summary_by_governorate.rds")

In [ ]:
# Summary by Education Level
summary_by_level <- school_data %>%
  group_by(level_name) %>%
  summarise(
    total_students = sum(nb_students),
    total_passed = sum(nb_passed),
    success_rate = round(total_passed / total_students * 100, 2),
    .groups = "drop"
  ) %>%
  arrange(desc(success_rate))

cat("📊 SUCCESS RATE BY EDUCATION LEVEL:\n")
cat("═══════════════════════════════════════\n")
print(summary_by_level)

# Save for Quarto
saveRDS(summary_by_level, "data/results/summary_by_level.rds")
cat("\n✅ Saved: data/results/summary_by_level.rds")

In [ ]:
# Statistical tests
cat("📊 STATISTICAL TESTS:\n")
cat("═══════════════════════════════════════\n\n")

# Normality test
shapiro_result <- shapiro.test(sample(school_data$success_rate, min(5000, nrow(school_data))))
cat("1️⃣ Shapiro-Wilk Normality Test:\n")
cat("   W =", round(shapiro_result$statistic, 4), "\n")
cat("   p-value =", format(shapiro_result$p.value, scientific = TRUE), "\n")
cat("   → Data is", ifelse(shapiro_result$p.value < 0.05, "NOT normally distributed", "normally distributed"), "\n\n")

# Kruskal-Wallis test (differences between governorates)
kruskal_result <- kruskal.test(success_rate ~ governorate, data = school_data %>% filter(!is.na(governorate)))
cat("2️⃣ Kruskal-Wallis Test (Governorate differences):\n")
cat("   Chi-squared =", round(kruskal_result$statistic, 2), "\n")
cat("   df =", kruskal_result$parameter, "\n")
cat("   p-value =", format(kruskal_result$p.value, scientific = TRUE), "\n")
cat("   → ", ifelse(kruskal_result$p.value < 0.05, "Significant differences between governorates!", "No significant differences"), "\n\n")

# Correlation
cor_result <- cor.test(school_data$nb_students, school_data$success_rate)
cat("3️⃣ Correlation (School Size vs Success Rate):\n")
cat("   r =", round(cor_result$estimate, 4), "\n")
cat("   p-value =", format(cor_result$p.value, scientific = TRUE), "\n")

# Save test results
stat_tests <- list(
  shapiro = shapiro_result,
  kruskal = kruskal_result,
  correlation = cor_result
)
saveRDS(stat_tests, "data/results/statistical_tests.rds")
cat("\n✅ Saved: data/results/statistical_tests.rds")

## 📊 Section 5: Visualizations - Educational Journey

In [ ]:
# Educational Journey: Year by Year Analysis
year_analysis <- school_data %>%
  mutate(
    year_order = case_when(
      str_detect(level_name, "سابعة") ~ 1,
      str_detect(level_name, "ثامنة") ~ 2,
      str_detect(level_name, "تاسعة") ~ 3,
      str_detect(level_name, "أولى") ~ 4,
      str_detect(level_name, "ثانية") ~ 5,
      str_detect(level_name, "ثالثة") ~ 6,
      TRUE ~ 0
    ),
    year_label = case_when(
      str_detect(level_name, "سابعة") ~ "7th\nBasic",
      str_detect(level_name, "ثامنة") ~ "8th\nBasic",
      str_detect(level_name, "تاسعة") ~ "9th\nBasic",
      str_detect(level_name, "أولى") ~ "1st\nSecondary",
      str_detect(level_name, "ثانية") ~ "2nd\nSecondary",
      str_detect(level_name, "ثالثة") ~ "3rd\nSecondary",
      TRUE ~ "Other"
    )
  ) %>%
  filter(year_order > 0) %>%
  group_by(year_order, year_label) %>%
  summarise(
    students = sum(nb_students),
    passed = sum(nb_passed),
    success_rate = round(passed / students * 100, 1),
    .groups = "drop"
  )

# Plot: Educational Journey
p_journey <- ggplot(year_analysis, aes(x = year_order, y = success_rate)) +
  geom_area(fill = "#3498db", alpha = 0.3) +
  geom_line(color = "#2c3e50", linewidth = 2) +
  geom_point(aes(size = students), color = "#e74c3c") +
  geom_text(aes(label = paste0(success_rate, "%")), vjust = -1.5, size = 5, fontface = "bold") +
  scale_x_continuous(breaks = year_analysis$year_order, labels = year_analysis$year_label) +
  scale_size_continuous(range = c(8, 18), guide = "none") +
  geom_hline(yintercept = 80, linetype = "dashed", color = "#e74c3c", alpha = 0.5) +
  labs(
    title = "📈 THE PATH TO BACCALAUREATE",
    subtitle = "Tracking success rates from 7th grade to 3rd Secondary (Pre-Bac)",
    x = "Educational Year", y = "Success Rate (%)"
  ) +
  ylim(70, 100) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 22, color = "#2c3e50"),
    plot.subtitle = element_text(hjust = 0.5, size = 14, color = "#7f8c8d")
  )

print(p_journey)

# Save plot
ggsave("img/01_educational_journey.png", p_journey, width = 14, height = 8, dpi = 150)
saveRDS(year_analysis, "data/results/year_analysis.rds")
cat("\n✅ Saved: img/01_educational_journey.png")

## 🎓 Section 6: Bac 2025 - Success by Field

In [ ]:
# Bac 2025 Success Rates by Field
p_bac_field <- ggplot(bac_by_field, aes(x = reorder(field_emoji, bac_success_rate), y = bac_success_rate)) +
  geom_col(aes(fill = bac_success_rate), width = 0.75, show.legend = FALSE) +
  geom_text(aes(label = paste0(bac_success_rate, "%")), hjust = -0.1, size = 6, fontface = "bold") +
  geom_hline(yintercept = 50, linetype = "dashed", color = "#e74c3c", linewidth = 1.2) +
  annotate("text", x = 4, y = 52, label = "← 50% Threshold", color = "#e74c3c", fontface = "bold", size = 5) +
  scale_fill_gradient2(low = "#c0392b", mid = "#f39c12", high = "#27ae60", midpoint = 50) +
  coord_flip() +
  ylim(0, 85) +
  labs(
    title = "🎓 BACCALAUREATE 2025 - SUCCESS BY FIELD",
    subtitle = "Official results: الدورة الرئيسية والمراقبة",
    x = "", y = "Success Rate (%)",
    caption = "Source: Ministry of Education, Tunisia"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 24, color = "#2c3e50"),
    plot.subtitle = element_text(hjust = 0.5, size = 14, color = "#7f8c8d"),
    axis.text.y = element_text(size = 14, face = "bold")
  )

print(p_bac_field)

# Save
ggsave("img/02_bac_by_field.png", p_bac_field, width = 14, height = 10, dpi = 150)
saveRDS(bac_by_field, "data/results/bac_by_field.rds")
cat("\n✅ Saved: img/02_bac_by_field.png")

## 🗺️ Section 7: Bac 2025 - Regional Rankings

In [ ]:
# Bac 2025 Success Rates by Governorate
p_bac_gov <- ggplot(bac_by_governorate, aes(x = reorder(governorate_ar, bac_success_rate), y = bac_success_rate)) +
  geom_segment(aes(xend = governorate_ar, yend = 40), color = "gray80", linewidth = 1) +
  geom_point(aes(color = bac_success_rate, size = candidates), show.legend = TRUE) +
  geom_text(aes(label = paste0(bac_success_rate, "%")), hjust = -0.3, size = 4, fontface = "bold") +
  scale_color_gradient2(low = "#c0392b", mid = "#f39c12", high = "#27ae60", midpoint = 55, name = "Success\nRate (%)") +
  scale_size_continuous(range = c(4, 12), name = "Candidates", labels = comma) +
  geom_hline(yintercept = 50, linetype = "dashed", color = "#e74c3c", linewidth = 1) +
  annotate("text", x = 8, y = 51.5, label = "← 50% Threshold", color = "#e74c3c", fontface = "bold") +
  coord_flip() +
  ylim(40, 78) +
  labs(
    title = "🏆 BAC 2025 - REGIONAL RANKING",
    subtitle = "ترتيب المندوبيات الجهوية حسب نسبة النجاح",
    x = "", y = "Success Rate (%)",
    caption = "Source: Ministry of Education - Bac 2025"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 22, color = "#2c3e50"),
    plot.subtitle = element_text(hjust = 0.5, size = 14, color = "#7f8c8d"),
    axis.text.y = element_text(size = 11, face = "bold"),
    legend.position = "right"
  )

print(p_bac_gov)

# Save
ggsave("img/03_bac_by_governorate.png", p_bac_gov, width = 14, height = 14, dpi = 150)
saveRDS(bac_by_governorate, "data/results/bac_by_governorate.rds")
cat("\n✅ Saved: img/03_bac_by_governorate.png")

## 💔 Section 8: The Reality Gap - Pre-Bac vs Actual Bac

In [ ]:
# Gap Analysis: Pre-Bac School Performance vs Actual Bac Results
comparison_data <- tibble(
  field = c("📚 Literature", "💼 Economics", "⚙️ Technical Sci.", "🔬 Exp. Sciences", 
            "💻 Computer Sci.", "🏃 Sports", "🔢 Mathematics"),
  prebac_estimate = c(90.5, 85.4, 88.2, 92.5, 94.1, 97.8, 97.2),
  bac_actual = c(23.02, 28.95, 35.02, 45.83, 48.47, 73.33, 74.93),
  field_ar = c("الآداب", "الاقتصاد", "العلوم التقنية", "العلوم التجريبية", 
               "علوم الإعلامية", "الرياضة", "الرياضيات")
) %>%
  mutate(gap = prebac_estimate - bac_actual)

# Reshape for plotting
comparison_long <- comparison_data %>%
  pivot_longer(cols = c(prebac_estimate, bac_actual), 
               names_to = "exam_type", values_to = "success_rate") %>%
  mutate(exam_label = if_else(exam_type == "prebac_estimate", "📚 Pre-Bac (School)", "🎓 Actual Bac 2025"))

# Plot: Gap Analysis
p_gap <- ggplot(comparison_long, aes(x = reorder(field, success_rate), y = success_rate, fill = exam_label)) +
  geom_col(position = "dodge", width = 0.7) +
  geom_text(aes(label = paste0(success_rate, "%")), 
            position = position_dodge(width = 0.7), hjust = -0.1, size = 4, fontface = "bold") +
  scale_fill_manual(values = c("📚 Pre-Bac (School)" = "#27ae60", "🎓 Actual Bac 2025" = "#c0392b"), name = "Exam Type") +
  coord_flip() +
  ylim(0, 110) +
  labs(
    title = "💔 THE REALITY GAP",
    subtitle = "Pre-Bac school success vs Actual Baccalaureate 2025 results",
    x = "", y = "Success Rate (%)"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 24, color = "#c0392b"),
    plot.subtitle = element_text(hjust = 0.5, size = 14, color = "#7f8c8d"),
    axis.text.y = element_text(size = 13, face = "bold"),
    legend.position = "bottom"
  )

print(p_gap)

# Save
ggsave("img/04_gap_analysis.png", p_gap, width = 14, height = 10, dpi = 150)
saveRDS(comparison_data, "data/results/gap_analysis.rds")
cat("\n✅ Saved: img/04_gap_analysis.png")

## 🔥 Section 9: Regional School Performance Heatmap

In [ ]:
# Heatmap: Pre-Bac by field and governorate
heatmap_data <- school_data %>%
  filter(str_detect(level_name, "ثالثة")) %>%
  mutate(
    field_short = case_when(
      str_detect(level_name, "آداب") ~ "Literature",
      str_detect(level_name, "رياضيات") ~ "Math",
      str_detect(level_name, "علوم تجريبية") ~ "Exp.Sci",
      str_detect(level_name, "علوم تقنية") ~ "Tech.Sci",
      str_detect(level_name, "علوم إعلامية") ~ "IT",
      str_detect(level_name, "إقتصاد") ~ "Economics",
      str_detect(level_name, "رياضة") ~ "Sports",
      TRUE ~ "Other"
    )
  ) %>%
  filter(!is.na(governorate)) %>%
  group_by(governorate, field_short) %>%
  summarise(success_rate = round(sum(nb_passed) / sum(nb_students) * 100, 1), .groups = "drop")

# Order governorates by overall success
gov_order <- heatmap_data %>%
  group_by(governorate) %>% summarise(avg = mean(success_rate)) %>%
  arrange(desc(avg)) %>% pull(governorate)

heatmap_data <- heatmap_data %>%
  mutate(governorate = factor(governorate, levels = rev(gov_order)))

# Plot heatmap
p_heatmap <- ggplot(heatmap_data, aes(x = field_short, y = governorate, fill = success_rate)) +
  geom_tile(color = "white", linewidth = 1.2) +
  geom_text(aes(label = success_rate, color = ifelse(success_rate > 88, "white", "black")), 
            size = 4, fontface = "bold") +
  scale_fill_gradient2(low = "#c0392b", mid = "#f39c12", high = "#27ae60", midpoint = 88, name = "Success\nRate (%)") +
  scale_color_identity() +
  labs(
    title = "🎯 PRE-BAC HEATMAP: Fields × Regions",
    subtitle = "Which fields struggle in which regions?",
    x = "Academic Field", y = "Governorate"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 22, color = "#2c3e50"),
    axis.text.x = element_text(angle = 45, hjust = 1, size = 12, face = "bold"),
    axis.text.y = element_text(size = 11, face = "bold")
  )

print(p_heatmap)

# Save
ggsave("img/05_heatmap.png", p_heatmap, width = 16, height = 14, dpi = 150)
saveRDS(heatmap_data, "data/results/heatmap_data.rds")
cat("\n✅ Saved: img/05_heatmap.png")

## 📊 Section 10: Success Distribution

In [ ]:
# Distribution of success rates
p_dist <- ggplot(school_data, aes(x = success_rate)) +
  geom_histogram(aes(y = after_stat(density)), bins = 30, fill = "#3498db", alpha = 0.7, color = "white") +
  geom_density(color = "#e74c3c", linewidth = 1.5) +
  geom_vline(xintercept = mean(school_data$success_rate), color = "#27ae60", linewidth = 1.5, linetype = "dashed") +
  annotate("text", x = mean(school_data$success_rate) + 3, y = 0.04, 
           label = paste0("Mean: ", round(mean(school_data$success_rate), 1), "%"),
           color = "#27ae60", fontface = "bold", size = 5) +
  labs(
    title = "📊 DISTRIBUTION OF SUCCESS RATES",
    subtitle = "How are school-level success rates distributed?",
    x = "Success Rate (%)", y = "Density"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold", hjust = 0.5, size = 20, color = "#2c3e50"),
    plot.subtitle = element_text(hjust = 0.5, size = 12, color = "#7f8c8d")
  )

print(p_dist)

# Save
ggsave("img/06_distribution.png", p_dist, width = 12, height = 8, dpi = 150)
cat("\n✅ Saved: img/06_distribution.png")

## 💾 Section 11: Export All Data for Quarto

In [ ]:
# Save all processed data for Quarto
saveRDS(school_data, "data/results/school_data_processed.rds")

# Final summary for Quarto
final_summary <- list(
  total_bac_candidates = sum(bac_by_governorate$candidates),
  total_bac_passed = sum(bac_by_governorate$passed),
  national_bac_rate = round(sum(bac_by_governorate$passed) / sum(bac_by_governorate$candidates) * 100, 2),
  best_field = "Mathematics (74.93%)",
  worst_field = "Literature (23.02%)",
  best_region = "Sfax 1 (71.31%)",
  worst_region = "Jendouba (43.08%)",
  biggest_gap = "Literature: 90% → 23% (-67 pts)"
)
saveRDS(final_summary, "data/results/final_summary.rds")

cat("═══════════════════════════════════════════════════════\n")
cat("✅ ALL DATA EXPORTED FOR QUARTO!\n")
cat("═══════════════════════════════════════════════════════\n\n")
cat("📁 Data files saved in: data/results/\n")
cat("   • school_data_processed.rds\n")
cat("   • summary_stats.rds\n")
cat("   • summary_by_governorate.rds\n")
cat("   • summary_by_level.rds\n")
cat("   • statistical_tests.rds\n")
cat("   • year_analysis.rds\n")
cat("   • bac_by_field.rds\n")
cat("   • bac_by_governorate.rds\n")
cat("   • gap_analysis.rds\n")
cat("   • heatmap_data.rds\n")
cat("   • final_summary.rds\n\n")
cat("📊 Plots saved in: img/\n")
cat("   • 01_educational_journey.png\n")
cat("   • 02_bac_by_field.png\n")
cat("   • 03_bac_by_governorate.png\n")
cat("   • 04_gap_analysis.png\n")
cat("   • 05_heatmap.png\n")
cat("   • 06_distribution.png\n\n")
cat("🎯 Now run Quarto to generate the website!")